In [ ]:
# neuron_classes = {}

# layers = {
#     "L1": [1],
#     "L23": [2, 3],
#     "L4": [4],
#     "L5": [5],
#     "L6": [6],
# }

# synapse_classes = ["EXC", "INH"]

# # Generate neuron classes for specific layers and synapse classes
# for hex_ind in range(77):
#     node_set = "hex" + str(hex_ind)
#     for layer_name, layer_numbers in layers.items():
#         for synapse_class in synapse_classes:
            
#             if (layer_name == 'L1') & (synapse_class == 'EXC'):
#                 continue
            
#             print(f"{layer_name}_{synapse_class}_{hex_ind}:")
#             print(f"  node_set: {node_set}")
#             print(f"  query:")
#             print(f"    layer: {layer_numbers}")
#             print(f"    synapse_class: {synapse_class}")
            


In [ ]:
# synapse_classes = ["EXC", "INH"]

# for hex_ind in range(77):
#     node_set = "hex" + str(hex_ind)
#     for synapse_class in synapse_classes:
#         print(f"ALL_{synapse_class}_{hex_ind}:")
#         print(f"  node_set: {node_set}")
#         print(f"  query:")
#         print(f"    synapse_class: {synapse_class}")
    

In [ ]:
import sys
sys.path.append('../../..')
import cortexetl as c_etl

ma = c_etl.analysis_initial_processing("5-FullCircuit-2-BetterMinis-FprScan.yaml", loglevel="ERROR")

a_AllCompartments = ma.AllCompartments_spikes

In [ ]:
# def get_xy_for_single_hex(hex_mean_flatspace_coords, hex_ind):

#     hex_x = hex_mean_flatspace_coords.etl.q(hex=hex_ind).iloc[0]['x']
#     hex_y = hex_mean_flatspace_coords.etl.q(hex=hex_ind).iloc[0]['y']

#     return hex_x, hex_y

# import pandas as pd
# hex_mean_flatspace_coords_path = '/gpfs/bbp.cscs.ch/project/proj83/home/isbister/data/reference_data_do_not_delete/hex_mean_flatspace_coords.parquet'
# c_etl.plot_evoked_hexes_by_hist(ma.hexes_spikes, hex_mean_flatspace_coords_path)

# for hex_ind in hex_inds:
#     hex_x, hex_y = get_xy_for_single_hex(hex_mean_flatspace_coords, hex_ind)

# Fig 11A1

In [ ]:
import numpy
import pandas
import os
import tqdm
import cortexetl as c_etl
from blueetl.parallel import call_by_simulation
import pandas as pd
from functools import partial
from matplotlib import pyplot as plt



def make_t_bins(t_start, t_end, t_step):
    t_bins = numpy.arange(t_start, t_end + t_step, t_step)
    return t_bins

def flatten_locations(locations, flatmap):
    if isinstance(flatmap, list):
        flat_locations = locations[flatmap].values
    else:
        from voxcell import VoxelData
        fm = VoxelData.load_nrrd(flatmap)
        flat_locations = fm.lookup(locations.values).astype(float)
        flat_locations[flat_locations == -1] = numpy.NaN
    return pandas.DataFrame(flat_locations, index=locations.index)


def make_spatial_bins(flat_locations, nbins=1000):
    mn = numpy.nanmin(flat_locations, axis=0)
    mx = numpy.nanmax(flat_locations, axis=0)
    ratio = (mx[1] - mn[1]) / (mx[0] - mn[0]) # ratio * nx ** 2 = nbins
    nx = int(numpy.sqrt(nbins / ratio))
    ny = int(nbins / nx)
    binsx = numpy.linspace(mn[0], mx[0] + 1E-3, nx + 1)
    binsy = numpy.linspace(mn[1], mx[1] + 1E-3, ny + 1)
    return binsx, binsy

def make_histogram_function(t_bins, loc_bins, location_dframe, spikes):
    t_step = numpy.mean(numpy.diff(t_bins))
    fac = 1000.0 / t_step
    nrns_per_bin = numpy.histogram2d(location_dframe.values[:, 0],
                                     location_dframe.values[:, 1],
                                     bins=loc_bins)[0]
    nrns_per_bin = nrns_per_bin.reshape((1,) + nrns_per_bin.shape)

    spikes = spikes.loc[numpy.in1d(spikes.values, location_dframe.index.values)]
    t = spikes.index.values
    loc = location_dframe.loc[spikes['gid']].values
    raw, _ = numpy.histogramdd((t, loc[:, 0], loc[:, 1]), bins=(t_bins,) + loc_bins)
    raw = fac * raw / (nrns_per_bin + 1E-6)
    return raw

def save(hist, t_bins, loc_bins, out_root):
    if not os.path.isdir(out_root):
        _ = os.makedirs(out_root)
    import h5py
    h5 = h5py.File(os.path.join(out_root, "spiking_activity_3d.h5"), "w")
    grp_bins = h5.create_group("bins")
    grp_bins.create_dataset("t", data=t_bins)
    grp_bins.create_dataset("x", data=loc_bins[0])
    grp_bins.create_dataset("y", data=loc_bins[1])

    grp_data = h5.create_group("histograms")
    for i, val in enumerate(hist.get()):
        grp_data.create_dataset("instance{0}".format(i), data=val)
    mn_data = numpy.mean(numpy.stack(hist.get(), -1), axis=-1)
    grp_data.create_dataset("mean", data=mn_data)
    return mn_data

def setup_cmap(hist, plotting_options, hist_mean=[]):
    
    hist_for_mask = hist
    if (len(hist_mean)):
        hist_for_mask = hist_mean
    
    masked_hist = hist
    mx_clim = numpy.percentile(hist, plotting_options['max_lim_pct'])
    mn_clim = numpy.percentile(hist, plotting_options['min_lim_pct'])
    indices_to_mask = numpy.asarray(numpy.argwhere(hist_for_mask <= plotting_options['mask_fr']))
    for pair in indices_to_mask:
        if (len(hist_mean)):
            masked_hist[:, pair[0], pair[1]] = numpy.nan
        else:
            masked_hist[pair[0], pair[1]] = numpy.nan
    
    
    cmap = plotting_options['cmap']
    cmap.set_bad('white',1.)
    
    clim = [mn_clim, mx_clim]
    
    return cmap, clim, masked_hist
    

def plot_and_save_single_image(hist, plotting_options, path):

    cmap, clim, masked_hist = setup_cmap(hist, plotting_options)
    
    fig = plt.figure()
    ax = fig.add_axes([0.05, 0.05, 0.9, 0.9])
    img = ax.imshow(masked_hist, cmap=cmap, clim=clim)
    plt.scatter(26.50221238938053, 67.87893675169182) #hex39
    plt.scatter(72.57363365352766, 41.2836038423319, c='r') #hex0
    plt.colorbar(img, cmap=cmap, label='FR (spikes / s')
    plt.box(False)
    plt.tick_params(left = False, right = False, labelleft = False, labelbottom = False, bottom = False)
    plt.savefig(path)
    plt.close()


import os
import numpy
import tqdm
import matplotlib
from matplotlib import pyplot as plt

#     plot(hist, t_bins, loc_bins, images_dir, flatspace_video_opt['delete_images'], flatspace_path_pre, plotting_options, hist_mean=hist_mean)
def plot(hist, t_bins, loc_bins, images_dir, delete_images, video_output_root, plotting_options, min_color_lim_pct=-1, hist_mean=[]):
    
    print(images_dir)
    
    if not os.path.isdir(images_dir):
        print('oh')
        _ = os.makedirs(images_dir)
    
#     print(plotting_options)
#     print(plotting_options['max_lim_pct'])
    cmap, clim, masked_hist = setup_cmap(hist, plotting_options, hist_mean=hist_mean)
    
    fps = []
    for t_start, t_end, bin_index in tqdm.tqdm(zip(t_bins[:-1], t_bins[1:], list(range(len(t_bins))))):
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_axes([0.05, 0.05, 0.9, 0.9])
        img = ax.imshow(masked_hist[bin_index, :, :], cmap=cmap, clim=clim)
        plt.colorbar(img, cmap=cmap, label='FR (spikes / s')
        
        ax.set_title("{0} - {1} ms".format(t_start, t_end))
        plt.box(False)
        plt.tick_params(left = False, right = False, labelleft = False, labelbottom = False, bottom = False)
        fn = "frame{:04d}.png".format(bin_index)
        fp = os.path.join(images_dir, fn)
        fig.savefig(fp)
        fps.append(fp)
        if (bin_index == 0):
            fn = "frame{:04d}.pdf".format(bin_index)
            fp = os.path.join(images_dir, fn)
            fig.savefig(fp)
        
        plt.close(fig)

    c_etl.video_from_image_files(fps, video_output_root + ".mp4")
    if delete_images:
        for f in fps:
            os.remove(f)

import numpy
from scipy.ndimage import gaussian_filter
def single_flatspace_video(simulation_row, 
                           filtered_dataframes, 
                           flat_locations, 
                           flatspace_video_opt, 
                           analysis_config,
                           plotting_options,
                           flatspace_path_pre=None, 
                           images_dir=None):

    window_row = filtered_dataframes['windows'].iloc[0]
    
    print(images_dir)

    if (flatspace_path_pre==None):
        flatspace_path_pre = flatspace_video_opt['video_output_root'] + str(simulation_row['simulation_id']) + "_" + simulation_row['simulation_string']
    if (images_dir==None):
        images_dir = str(window_row['flatspace_video_images_dir']) + "/" + flatspace_video_opt['vid_str'] + "_" + str(simulation_row['simulation_id']) + "/"
    

    t_bins = make_t_bins(window_row['t_start'], window_row['t_stop'], flatspace_video_opt['t_step'])
    spikes = filtered_dataframes['spikes'].loc[:, ['time', 'gid']].set_index('time')
    
    loc_bins = make_spatial_bins(flat_locations, flatspace_video_opt['n_spatial_bins'])
    spatial_temporal_hist = make_histogram_function(t_bins, loc_bins, flat_locations, spikes)
    smoothed_spatial_temporal_hist = gaussian_filter(spatial_temporal_hist, [flatspace_video_opt['temporal_smoothing_sigma'], 1.0, 1.0])
    hist = smoothed_spatial_temporal_hist
    hist_mean = numpy.mean(hist, axis=0)
    
    plot(hist, t_bins, loc_bins, images_dir, flatspace_video_opt['delete_images'], flatspace_path_pre + '_spont', plotting_options)
    
    plot_and_save_single_image(hist_mean, plotting_options, flatspace_path_pre + '_hist_mean.pdf')


    r_dict = {"smoothed_spatial_temporal_hist": smoothed_spatial_temporal_hist,
            "t_bins": t_bins}
    return r_dict


import os
from blueetl.parallel import call_by_simulation
from functools import partial

a = a_AllCompartments

for flatspace_video_key in a.analysis_config.custom['flatspace_videos']:
    flatspace_video_opt = a.analysis_config.custom['flatspace_videos'][flatspace_video_key]
    flatspace_video_opt['vid_str'] = flatspace_video_opt['window'] + "_" + str(flatspace_video_opt['t_step']) + "_" + str(flatspace_video_opt['n_spatial_bins']) + "_" + str(flatspace_video_opt['temporal_smoothing_sigma'])
    flatspace_video_opt['video_output_root'] = str(a.figpaths.flatspace_videos) + "/" + flatspace_video_opt['vid_str'] + "/"
    os.makedirs(flatspace_video_opt['video_output_root'], exist_ok=True)

    dataframes={
        "circuits": a.repo.simulations.df.loc[:, ['circuit', 'circuit_id', 'simulation_id']],
        "spikes": a.repo.report.df.etl.q(neuron_class="ALL", window=flatspace_video_opt['window']),
        "windows": a.repo.windows.df.etl.q(window=flatspace_video_opt['window']), 
        "neurons": a.repo.neurons.df.etl.q(neuron_class="ALL")}

    gids = a.repo.neurons.df.etl.q(circuit_id=0)['gid']
    locations = a.repo.simulations.df.loc[:, ['circuit', 'circuit_id', 'simulation_id']].iloc[0]['circuit'].nodes[None].get(gids, ["x", "y", "z"])
    flat_locations = c_etl.flatten_locations(locations, a.analysis_config.custom["flatmap"])

    results = call_by_simulation(a.repo.simulations.df.etl.q(ca=1.05, depol_stdev_mean_ratio=0.4, desired_connected_proportion_of_invivo_frs=0.2), 
                                    dataframes, 
                                    func=partial(single_flatspace_video, 
                                                flat_locations=flat_locations, 
                                                flatspace_video_opt=flatspace_video_opt, 
                                                analysis_config=a.analysis_config.custom,
                                                plotting_options={"cmap": matplotlib.cm.cividis,
                                                                  "mask_fr": 0.0001,
                                                                 "max_lim_pct": 99,
                                                                 "min_lim_pct": 0},
                                                flatspace_path_pre=str(a.figpaths.root) + '/' + 'Fig7D-correlated_flatspace', 
                                                images_dir=str(a.figpaths.root) + '/' + 'Fig7D-correlated_flatspace/'),
                                    how='series')

# Fig 11B

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import pandas as pd
import time
import os
from matplotlib.ticker import MultipleLocator
import seaborn as sns
from functools import partial

from blueetl.constants import *
from blueetl.parallel import call_by_simulation


import cortexetl as c_etl



def plot_simulation_raster(simulation_row, filtered_dataframes, analysis_config, raster_option_combination):
        
    plot_raster(simulation_row, 
                        filtered_dataframes['windows'].iloc[0], 
                        filtered_dataframes['spikes'], 
                        filtered_dataframes['neurons'], 
                        filtered_dataframes['neuron_classes'], 
                        raster_option_combination,
                        analysis_config,
                        simulation_histograms=filtered_dataframes['histograms'])


def plot_rasters(a, custom_file_path=None, simulation_filter={}):

    a.repo.simulations.df["SummaryPNG"] = a.repo.simulations.df['rasters_dir'] / (a.repo.simulations.df['simulation_string'].astype(str) + "_SUMMARY.png")

    for dict_to_unpack in a.analysis_config.custom['raster_windows']:

        window_str = list(dict_to_unpack.keys())[0]
        fig_dims = list(dict_to_unpack.values())[0]

        if (type(fig_dims) == list):
            fig_width = fig_dims[0]
            fig_height = fig_dims[1]
        else:
            fig_width = fig_dims
            fig_height = fig_width * 0.6
        
        raster_option_combinations = [
                                    RasterOptions(a, window_str=window_str, neuron_group_y_axis_equal=False, use_spikes=True, smoothing_type='Gaussian', hist_bin_size=3.0, kernel_sd=1.0, neuron_classes=c_etl.LAYER_EI_NEURON_CLASSES, neuron_class_groupings=c_etl.NEURON_CLASS_NO_GROUPINGS, fig_width=fig_width, fig_height=fig_height, custom_file_path=custom_file_path),
                                    RasterOptions(a, window_str=window_str, neuron_group_y_axis_equal=True, use_spikes=False, smoothing_type='Gaussian', hist_bin_size=3.0, kernel_sd=1.0, neuron_classes=['ALL'], neuron_class_groupings=[['ALL']], extra_string='All', fig_width=fig_width, fig_height=fig_height, custom_file_path=custom_file_path),
                                    RasterOptions(a, window_str=window_str, neuron_group_y_axis_equal=True, use_spikes=False, smoothing_type='Gaussian', hist_bin_size=3.0, kernel_sd=1.0, neuron_classes=c_etl.LAYER_EI_NEURON_CLASSES + ['ALL_EXC', 'ALL_INH'], neuron_class_groupings=c_etl.LAYER_EI_NEURON_CLASS_GROUPINGS, extra_string='LayerEI', fig_width=fig_width, fig_height=fig_height, custom_file_path=custom_file_path),
                                    RasterOptions(a, window_str=window_str, neuron_group_y_axis_equal=True, use_spikes=False, smoothing_type='Gaussian', hist_bin_size=5.0, kernel_sd=1.0, neuron_classes=c_etl.LAYER_EI_NEURON_CLASSES + ['ALL_EXC', 'ALL_INH'], neuron_class_groupings=c_etl.LAYER_EI_NEURON_CLASS_GROUPINGS, extra_string='LayerEI', fig_width=fig_width, fig_height=fig_height, custom_file_path=custom_file_path, lw=0.4, seperator_lw=0.2)
                                    ]

        if (a.analysis_config.custom['plot_rasters']):

            print(f"\n----- Plot rasters, window: {window_str} -----")

            for roc in raster_option_combinations:

                dataframes={
                        "spikes": a.repo.report.df.etl.q(neuron_class=roc.neuron_classes, window=roc.window_str),
                        "windows": a.repo.windows.df.etl.q(window=roc.window_str), 
                        "neurons": a.repo.neurons.df,
                        "neuron_classes": a.repo.neuron_classes.df.etl.q(neuron_class=roc.neuron_classes),
                        "histograms": a.features.histograms.df.etl.q(neuron_class=roc.neuron_classes, window=roc.window_str, bin_size=roc.hist_bin_size, smoothing_type=roc.smoothing_type, kernel_sd=roc.kernel_sd)
                        }
            
                results = call_by_simulation(a.repo.simulations.df.etl.q(simulation_filter), 
                                                dataframes, 
                                                func=partial(plot_simulation_raster, analysis_config=a.analysis_config.custom, raster_option_combination=roc), how="series")

        if (a.analysis_config.custom['create_raster_videos']):

            print(f"\n----- Create raster videos, window: {window_str} -----")

            for roc in raster_option_combinations:
                if (a.repo.windows.df.etl.q(window=window_str).iloc[0]['window_type'] == "spontaneous"):
                    for mask_key in ['', 'bursting', 'bursting_or_fr_above_threshold_or_ei_corr_r_out_of_range']:
                        roc.create_video(a, mask_key=mask_key)

                elif (a.repo.windows.df.etl.q(window=window_str).iloc[0]['window_type'] == "evoked_stimulus_onset_zeroed"):
                    for mask_key_and_invert_mask_bool in [['', False], ['overly_sustained_response', False], ['overly_sustained_response', True], ['higher_secondary_peak', False], ['higher_secondary_peak', True], ['too_much_trial_to_trial_variance', False], ['too_much_trial_to_trial_variance', True], ['evoked_mask', False], ['evoked_mask', True]]:
                        roc.create_video(a, mask_key=mask_key_and_invert_mask_bool[0], invert_mask=mask_key_and_invert_mask_bool[1])




class RasterOptions(object):

    def __init__(self, a, window_str='', neuron_group_y_axis_equal=True, use_spikes=True, smoothing_type='', hist_bin_size=3.0, kernel_sd=1.0, neuron_classes=[], neuron_class_groupings=[], extra_string='', fig_width=20, fig_height=15, custom_file_path=None, lw=0.2, seperator_lw=0.2):

        self.window_str = window_str
        self.neuron_group_y_axis_equal = neuron_group_y_axis_equal
        self.use_spikes = use_spikes
        self.smoothing_type = smoothing_type
        self.hist_bin_size = hist_bin_size
        self.kernel_sd = kernel_sd
        self.neuron_classes = neuron_classes
        self.neuron_class_groupings = neuron_class_groupings
        self.extra_string = extra_string
        self.fig_width = fig_width
        self.fig_height = fig_height
        self.lw = lw
        self.seperator_lw = seperator_lw

        
        windows = pd.merge(a.repo.simulations.df.reset_index().drop(["index"], axis=1), a.repo.windows.df.reset_index()).set_index(['index'])
        if (custom_file_path == None):
            custom_file_path = windows['rasters_dir'].astype(str)

        self.options_str = "{spikes}".format(spikes="S_" if use_spikes == True else "NS_") + smoothing_type + "_" + str(hist_bin_size) + "_" + str(kernel_sd) + "_" + "{yax}".format(yax="YNE_" if neuron_group_y_axis_equal == True else "YE_") + extra_string
        self.df_file_path_key = self.options_str + '_rasters_path_png'
        self.df_file_path_pdf_key = self.options_str + '_rasters_path_pdf'

        a.repo.windows.df.loc[windows.index, self.df_file_path_key] = custom_file_path + (windows['window'].astype(str) + "_" + self.options_str + "_RASTER.png")
        a.repo.windows.df.loc[windows.index, self.df_file_path_pdf_key] = custom_file_path+ (windows['window'].astype(str) + "_" + self.options_str + "_RASTER.pdf")


    def create_video(self, a, mask_key='', invert_mask=False):

        windows_df = a.repo.windows.df.etl.q(window=self.window_str)
        
        if (mask_key != ''):
            windows_with_stats_df = pd.merge(windows_df, a.custom['custom_simulations_post_analysis'])
            q = {mask_key: invert_mask}
            windows_df = windows_with_stats_df.etl.q(q)

        raster_videos_window_dir = str(a.figpaths.raster_videos) + "/" + self.window_str + "/" + mask_key + str(invert_mask) + "/"
        os.makedirs(raster_videos_window_dir, exist_ok=True)

        video_fn = raster_videos_window_dir + self.window_str + "_" + self.options_str + '_' + mask_key + ":" + str(invert_mask) + ".mp4"

        c_etl.video_from_image_files(windows_df[self.df_file_path_key].astype(str).tolist(), video_fn)


def renormalise_psth(psth):
    new_hist = psth  - np.min(psth)
    new_hist = new_hist / np.max(new_hist)
    return new_hist

def plot_raster(simulation_row, window_row, window_spikes, circuit_neurons, neuron_classes, raster_option_combination, analysis_config, simulation_histograms=None, spont_ei_corr_rval=-5.0):

    sns.set(style="ticks", context="paper", font="Helvetica Neue",
        rc={"axes.labelsize": 7, "legend.fontsize": 6, "axes.linewidth": 0.6, "xtick.labelsize": 6, "ytick.labelsize": 6,
            "xtick.major.size": 2, "xtick.major.width": 0.5, "xtick.minor.size": 1.5, "xtick.minor.width": 0.3,
            "ytick.major.size": 2, "ytick.major.width": 0.5, "ytick.minor.size": 1.5, "ytick.minor.width": 0.3,
            "axes.titlesize": 7, "axes.spines.right": False, "axes.spines.top": False})

    start_time = time.time()
    plt.figure(figsize=(raster_option_combination.fig_width, raster_option_combination.fig_height))
    ax = plt.gca()

    # SET NEURON CLASS COLOURS
    neuron_classes = neuron_classes.copy()
    neuron_classes.loc[:, 'c'] = neuron_classes.apply(lambda row : c_etl.NEURON_CLASS_LAYERS_AND_SYNAPSE_CLASSES[row['neuron_class']]["color"], axis=1)

    # SET NEURON CLASS START 
    if (not raster_option_combination.neuron_group_y_axis_equal):
        neuron_classes['cum_sum'] = neuron_classes[COUNT].cumsum()
        neuron_classes['start_pos'] = neuron_classes['cum_sum'].shift().fillna(0)
    else:
        neuron_classes['start_pos'] = 0
        neuron_classes['cum_sum'] = 0
        for neuron_class_index, neuron_class in neuron_classes.iterrows():            
            for neuron_class_grouping_index, neuron_class_grouping in enumerate(raster_option_combination.neuron_class_groupings):
                if neuron_class["neuron_class"] in neuron_class_grouping:
                    neuron_classes.loc[neuron_class_index, 'start_pos'] = neuron_class_grouping_index * 5000
                    neuron_classes.loc[neuron_class_index, 'cum_sum'] = neuron_class_grouping_index * 5000 + 5000


    # PLOT SPIKES
    if (raster_option_combination.use_spikes):
        window_spikes = pd.merge(neuron_classes, window_spikes)
        window_spikes = window_spikes.set_index([CIRCUIT_ID, NEURON_CLASS, GID])
        circuit_neurons = circuit_neurons.set_index([CIRCUIT_ID, NEURON_CLASS, GID])
        window_spikes = circuit_neurons.join(window_spikes, how='inner')

        shuffled_within_neuron_class = True
        if (shuffled_within_neuron_class):
            for neuron_class_index, neuron_class in neuron_classes.iterrows(): 
                nc_w_spikes = window_spikes.etl.q(neuron_class=neuron_class[NEURON_CLASS])
                nc_random_map = np.arange(neuron_class[COUNT])
                np.random.shuffle(nc_random_map)
                shuffled_neuron_class_indices = nc_w_spikes.neuron_class_index.map(lambda x: nc_random_map[x])
                neuron_scatter_pos = nc_w_spikes['start_pos'] + shuffled_neuron_class_indices
                ax.scatter(nc_w_spikes[TIME], neuron_scatter_pos, s=0.1, c='k', linewidths=0) #, facecolors='c', s=0.2, c=nc_w_spikes['c']

        else:
            neuron_scatter_pos = window_spikes['start_pos'] + window_spikes['neuron_class_index']
            ax.scatter(window_spikes[TIME], neuron_scatter_pos, s=0.1, c=window_spikes['c'], linewidths=0)



    # OPTIONALLY LOAD INVIVO_HISTOGRAMS
    if (window_row['window_type'] in ['evoked_stimulus_onset_zeroed', 'evoked_cortical_onset_zeroed']):
        vivo_df = pd.read_feather(analysis_config['vivo_df']).reset_index()
        vivo_neuron_classes = vivo_df["neuron_class"].unique()

    # PLOT DIVIDERS AND HISTOGRAMS
    for neuron_class_index, neuron_class in neuron_classes.iterrows():
        plt.plot([window_row['t_start'], window_row['t_stop']], [neuron_class['start_pos'], neuron_class['start_pos']], lw=raster_option_combination.seperator_lw, c='k')

        if (simulation_histograms is not None):

            bin_indices, hist_array = c_etl.hist_elements(simulation_histograms.etl.q(simulation_id=window_row[SIMULATION_ID], 
                                                                                        neuron_class=neuron_class[NEURON_CLASS], 
                                                                                        window=raster_option_combination.window_str, 
                                                                                        bin_size=raster_option_combination.hist_bin_size, 
                                                                                        smoothing_type=raster_option_combination.smoothing_type, 
                                                                                        kernel_sd=raster_option_combination.kernel_sd))


            if (hist_array.shape[0] != 0):
                hist_max = np.max(hist_array)
                if (hist_max != 0.0):
                    max_normalised_hist = hist_array / np.max(hist_array)

                    if (not raster_option_combination.neuron_group_y_axis_equal):
                        plt.plot(window_row['t_start'] + (bin_indices * raster_option_combination.hist_bin_size), neuron_class['cum_sum'] - neuron_class[COUNT]*max_normalised_hist, c=neuron_class['c'], lw=raster_option_combination.lw)
                    else:
                        plt.plot(window_row['t_start'] + (bin_indices * raster_option_combination.hist_bin_size), neuron_class['cum_sum'] - 5000*(max_normalised_hist), c=neuron_class['c'], lw=raster_option_combination.lw)

                    if (window_row['window_type'] in ['evoked_stimulus_onset_zeroed', 'evoked_cortical_onset_zeroed']):
                        if (neuron_class[NEURON_CLASS] in list(c_etl.vivo_neuron_class_map.keys())):
                            in_vivo_neuron_class = c_etl.vivo_neuron_class_map[neuron_class[NEURON_CLASS]]
                            if in_vivo_neuron_class in vivo_neuron_classes:
                                nc_data = vivo_df[(vivo_df["neuron_class"] == in_vivo_neuron_class) & (vivo_df["barrel"] == "C2")].iloc[0]
                                nc_mean = nc_data["psth_mean"]
                                # nc_sd = nc_data["psth_sd"]

                                x = window_row['t_start'] - 50.0 + (1.0 * np.asarray(range(len(nc_mean))))

                                if (not raster_option_combination.neuron_group_y_axis_equal):
                                    y = neuron_class['cum_sum'] - neuron_class[COUNT]*nc_mean
                                else:
                                    y = neuron_class['cum_sum'] - 5000*(nc_mean)


                                plt.plot(x, y, c=neuron_class['c'], lw=raster_option_combination.lw, linestyle='--')




    # PLOT OPTIONS AND SAVE
    x_tick_distance = 5
    duration = window_row['t_stop'] - window_row['t_start']
    if (duration > 100):
        x_tick_distance = 100
    if (duration > 1000):
        x_tick_distance = 1000

    ax.set_yticks(neuron_classes['start_pos'] + (neuron_classes[COUNT]/2.0), minor=False)
    if (raster_option_combination.use_spikes):
        ax.set_yticklabels([c_etl.neuron_class_label_map[nc] for nc in neuron_classes['neuron_class']], minor=False)
    else:
        ax.set_yticks(neuron_classes['start_pos'] + (5000.0/2.0), minor=False)
#         print([nc_str.split('_')[0] for nc_str in neuron_classes['neuron_class']])
        ax.set_yticklabels([nc_str.split('_')[0] for nc_str in neuron_classes['neuron_class']], minor=False)

    ax.set_xlim([window_row['t_start'], window_row['t_stop']])
    ax.set_ylim([0, neuron_classes['cum_sum'].max()])
    ax.set_ylabel('')
    # ax.set_xlabel('Time from window start (ms)')
    ax.set_xlabel('Time (ms)')
    ax.set_axisbelow(True)
    title_str = str(simulation_row['simulation_id']) + " " + simulation_row['simulation_string'] #+ "    " + raster_option_combination.options_str
    if (spont_ei_corr_rval != -5.0):
        title_str += "  spont_ei_corr_rval: " + str(np.around(spont_ei_corr_rval, decimals=3))
    ax.set_title(title_str)

    ax.xaxis.set_major_locator(MultipleLocator(x_tick_distance))
    ax.xaxis.set_minor_locator(MultipleLocator(x_tick_distance))
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.invert_yaxis()

    plt.savefig(window_row[raster_option_combination.df_file_path_key], bbox_inches='tight')
    plt.savefig(window_row[raster_option_combination.df_file_path_pdf_key], bbox_inches='tight')
    plt.close()

    print("Raster generated: ", "{:.2f}".format(time.time() - start_time), 's')


ma.hex0_spikes.analysis_config.custom['plot_rasters'] = True
ma.hex59_spikes.analysis_config.custom['plot_rasters'] = True
    
ma.hex0_spikes.analysis_config.custom['raster_windows'][0]['conn_spont'] = [2.5, 2.13]
ma.hex59_spikes.analysis_config.custom['raster_windows'][0]['conn_spont'] = [2.5, 2.13]
# print(ma.hex0_spikes.repo.simulations.df.etl.q(desired_connected_proportion_of_invivo_frs=0.15))
# print(ma.hex0_spikes.repo.spikes.df.etl.q(desired_connected_proportion_of_invivo_frs=0.15))
plot_rasters(ma.hex0_spikes, custom_file_path=str(a.figpaths.root) + '/' + 'Fig11B-Hex0', simulation_filter={"simulation_id":3})
plot_rasters(ma.hex59_spikes, custom_file_path=str(a.figpaths.root) + '/' + 'Fig11B-Hex59', simulation_filter={"ca":1.05, "depol_stdev_mean_ratio": 0.4, "desired_connected_proportion_of_invivo_frs": 0.15})
# c_etl.plot_rasters(a_hex0, custom_file_path='figures/Fig7A-HexX', simulation_filter={"ca":1.1, "depol_stdev_mean_ratio": 0.2, "desired_connected_proportion_of_invivo_frs": 0.9})

In [ ]:
c_etl.multi_hex_analysis(a_AllCompartments)

In [ ]:
def plot_evoked_hexes_by_hist(a, hex_mean_flatspace_coords):

    hists_for_plot = a.features.histograms.df.etl.q(window="evoked_SOZ_100ms", bin_size=1.0, smoothing_type='Gaussian', kernel_sd=1.0).reset_index()
    for simulation_id in hists_for_plot.simulation_id.unique():
        simulation_hists = hists_for_plot.etl.q(simulation_id=simulation_id)
        plot_evoked_hists_by_hex(a, simulation_id, simulation_hists, list(range(0, 77)), hex_mean_flatspace_coords, 0)

import pandas as pd
hex_mean_flatspace_coords_path = '/gpfs/bbp.cscs.ch/project/proj83/home/isbister/data/reference_data_do_not_delete/hex_mean_flatspace_coords.parquet'
hex_mean_flatspace_coords = pd.read_parquet(hex_mean_flatspace_coords_path)
c_etl.plot_evoked_hexes_by_hist(ma.hexes_spikes, hex_mean_flatspace_coords)